## Question 1

In [0]:

fileloc = "/FileStore/tables/users.csv"
filetype ="csv"

infer_schema ="false"
firstrow_header ="false"
delimiter =","

df = spark.read.format(filetype).option("inferSchema",infer_schema).option("header",firstrow_header).option("sep",delimiter).load(fileloc)

display(df)

In [0]:
dff_user = df.withColumnRenamed("_c0","UserID").withColumnRenamed("_c1","EmailID").withColumnRenamed("_c2","NativeLanguage").withColumnRenamed("_c3","Location")
       

dff_user.show()

In [0]:
fileloc = "/FileStore/tables/transactions.csv"
filetype ="csv"

infer_schema ="false"
firstrow_header ="false"
delimiter =","

df_Transaction = spark.read.format(filetype).option("inferSchema",infer_schema).option("header",firstrow_header).option("sep",delimiter).load(fileloc)

dff_Transaction = df_Transaction.withColumnRenamed("_c0","Transaction_ID").withColumnRenamed("_c1","Product_ID").withColumnRenamed("_c2","UserID").withColumnRenamed("_c3","Price").withColumnRenamed("_c4","Product_Description")
       

dff_Transaction.show()


In [0]:
dff_user.createOrReplaceTempView("User_Master")

sqlDF_USER = spark.sql("SELECT * FROM User_Master")
#sqlDF_USER.show()

dff_Transaction.createOrReplaceTempView("Transaction_Data")

sqlDF_Transaction = spark.sql("SELECT * FROM Transaction_Data")
sqlDF_Transaction.show()



## Question 1a

In [0]:


sqlContext.sql("SELECT count( distinct UM.Location) as CountLocation FROM Transaction_Data TD Inner join User_Master UM on TD.UserID = UM.USERID ").show()


## Question 1b

In [0]:

sqlContext.sql("SELECT UserID as User ,Product_Description FROM Transaction_Data  order by cast(UserID as int) asc ").show()


## Question 1C

In [0]:
# 1C

sqlContext.sql("SELECT UserID as User ,Product_Description , sum(Price) as Price_perProduct FROM Transaction_Data group by UserID ,Product_Description   order by cast(UserID as int) asc ").show()


## Question 2
Our motive is to recommend similar rated movies to each user.

In [0]:
import sys
from pyspark import SparkConf, SparkContext, SparkFiles 
from math import sqrt 
import os

#importing necessary python and pySpark modules such as sys (to access various functions and variables used in Python runtime environment) , sparkConf ( Configuration for a Spark Application) ,SparkContext(Main Entry point for Spark functionality) , SparkFiles( resolves path to files added through sc.addFile() ) , math ( to access square root function from python math module ) , os (module that provides function to interact with operating system) for the script to execute.

In [0]:
# This is  a user defined function which provides functionality to list movienames and its ID.

def loadMovieNames(): # Initialization or defination of function
    movieNames = {}  # Creation of an empty dictionary names movieNames
    spark.sparkContext.addFile("dbfs:/FileStore/shared_uploads/sbsnigdhac@gmail.com/itemfile.txt") # Using SparkContext we are adding a file in Spark cluster 
    with open(SparkFiles.get('itemfile.txt'),encoding="ISO-8859-1") as f: # Accessing the file in Spark Job using SparkFile.get() and is being assigned to f
        for line in f:  # for loop is used to access each element in f and getting assigned to line variable.
            fields = line.split('|') # spliting each element of variable line using the pipe delimiter "|" and adding it to the list fields.
            movieNames[int(fields[0])] = fields[1] # now movie id (0th index of list fields ) and movie name (1st index of list fields) are being added into the dictionary as key (movie id) - value (movie name)
    return movieNames # this is the return statement of this function, which returns the dictionary moviewNames consisting of movie id and movie name .
	
loadMovieNames() # calling the function to check the output



Out[43]: {1: 'Toy Story (1995)',
 2: 'GoldenEye (1995)',
 3: 'Four Rooms (1995)',
 4: 'Get Shorty (1995)',
 5: 'Copycat (1995)',
 6: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 7: 'Twelve Monkeys (1995)',
 8: 'Babe (1995)',
 9: 'Dead Man Walking (1995)',
 10: 'Richard III (1995)',
 11: 'Seven (Se7en) (1995)',
 12: 'Usual Suspects, The (1995)',
 13: 'Mighty Aphrodite (1995)',
 14: 'Postino, Il (1994)',
 15: "Mr. Holland's Opus (1995)",
 16: 'French Twist (Gazon maudit) (1995)',
 17: 'From Dusk Till Dawn (1996)',
 18: 'White Balloon, The (1995)',
 19: "Antonia's Line (1995)",
 20: 'Angels and Insects (1995)',
 21: 'Muppet Treasure Island (1996)',
 22: 'Braveheart (1995)',
 23: 'Taxi Driver (1976)',
 24: 'Rumble in the Bronx (1995)',
 25: 'Birdcage, The (1996)',
 26: 'Brothers McMullen, The (1995)',
 27: 'Bad Boys (1995)',
 28: 'Apollo 13 (1995)',
 29: 'Batman Forever (1995)',
 30: 'Belle de jour (1967)',
 31: 'Crimson Tide (1995)',
 32: 'Crumb (1994)',
 33: 'Desperado (1995)

In [0]:
# The defination of this function is to provide similiarities between pair of movies based on the ratings by each user.

def makePairs(user_ratings): # Initialization or a defination of a function . This is a parameterized function, which takes tuples as a parameter
    user, ratings=user_ratings #this line assigns the 1st element of the tuple to user variable and second element of the parameter tuple to ratings variable. By unpacking tuple, function has created created user ID and ratings list.
    (movie1, rating1) = ratings[0] # The ratings list contains two elements or tuple each providing movie name/movie ID and ratings of respective movie again by using tuple unpacking.
    (movie2, rating2) = ratings[1] # similarly the second element or tuple of the ratings list is assigned to movie2 and rating2 .
    return ((movie1, movie2), (rating1, rating2)) # finally it is returning the obtained pair of movie and ratings .

# To compute cosine similarity metrics on pair of movies , this functio has created tuple of movie pair and of their ratings.

In [0]:
# This user defined tuple parameterized function is used to filter out duplicates from the combinations created in the above function code. the defination of the function is based on the logic to remove combinations such as (movie1, movie1) , (movie2, movie2) and (movie2, movie1).

def filterDuplicates(userID_ratings): # This is the initialization or defination step of the function. Its a parameterized function that accepts tuple as a parameter.
    userID, ratings = userID_ratings # this line unfolds the parameter tuple containing information about the userid and ratings. It assign userid to the userid variable and ratings provided by the user to the ratings list
    (movie1, rating1) = ratings[0] # The ratings list contains two tuples each containg movieid and ratings , which inturn is getting assigned to the tuple .
    (movie2, rating2) = ratings[1] # assigning second element of the ratings list to this tuple.
    return movie1 < movie2 # As the purpose of the function was to filter out the duplicates [(movie2, movie1) = (movie1, movie2) , (movie1, movie1) , (movie2, movie2) ] , it is returning those combinations satisfying movie1 id < movie2 id.



In [0]:
# The purpose of the function is to calculate cosine similarity score between two movies on the basis on ratings provided by users.

def computeCosineSimilarity(ratingPairs): # This is the function defination and this functiona accepts tuple as parameter. The paremeter is the two ratings Rating A and B given by two users for the same movie.
    numPairs = 0 # Initializing variable to zero
    sum_xx = sum_yy = sum_xy = 0 # Initializing variable to zero
    for ratingX, ratingY in ratingPairs: # This iterates the over the pair of ratings provided by the function parameter
        sum_xx += ratingX * ratingX # It cumulatively adds square of one rating and assign to the variable.
        sum_yy += ratingY * ratingY # It cumulatively adds square of another rating and assign to the variable.
        sum_xy += ratingX * ratingY # It cumulatively adds  the product of both ratings and assign to the variable.
        numPairs += 1 # updates the number of pairs in each iteration.
    numerator = sum_xy # To calculate the cosine similarity score numerator (sum_xy) / denominator (sqrt (sum_xx)* sqrt (sum_yy)) , it is defining numerator
    denominator = sqrt(sum_xx) * sqrt(sum_yy) # similarly defining denominator
    score = 0  # inittilaizing the score = 0
    if (denominator):
        score = (numerator / (float(denominator))) # This line says if denominator <> 0 then calculate this block.
    return (score, numPairs) # this returns the cosine similarity score and number of user pairs contributed in the score 

In [0]:
if __name__ == '__main__': # this line represents that the code should be executed only if its is run directly and not by imported as module.
    conf = SparkConf().setMaster("local[*]").setAppName("MovieSimilarities") #SparkConf() creates Spark Configuarion object and setMaster sstaes that Spark should run in local mode using as many worker threads as available processors. setAppName setting the Spark application name as MovieSimilarities
    print("\nLoading movie names...")  # prints a message.
    nameDict = loadMovieNames() # calling the first function which returns dictionary of movie names and assign it to a dictionary variable.
    
    data = sc.textFile("dbfs:/FileStore/shared_uploads/sbsnigdhac@gmail.com/datafile2-1.txt") # Read text file using Spark Context.
    ratings = data.map(lambda l: l.split()).map(lambda l: (int(l[0]), (int(l[1]), float(l[2])))) # split the lines using space and then map each line to a tuple (userid , movieid , rating)
    joinedRatings = ratings.join(ratings) # self joining so that it creates all possible combination of the ratings and movies watched by same user .
    uniqueJoinedRatings = joinedRatings.filter(filterDuplicates) # calculating unique pair by filetring out duplicates.
    moviePairs = uniqueJoinedRatings.map(makePairs) # movie rating pair
    moviePairRatings = moviePairs.groupByKey() # grouping the data by key i.e. by movieid
    moviePairSimilarities = moviePairRatings.mapValues(computeCosineSimilarity).cache() #applying cosine similarity function on pairs and the results are cached so that it can be accessed fastly in future references.

    if (len(sys.argv) >=0):  # Checking the length of the command whether it is greater than zero.
        scoreThreshold = 0.10 # we are setting this threshold to filter out top 10 movies.
        coOccurenceThreshold = 2 # To filter out movies that are occuring once i.e. co-occurrence of  movies with less than 2
        movieID =1 # the first id in the dataset is 1 , hence.
        filteredResults = moviePairSimilarities.filter(lambda x: (x[0][0] == movieID or x[0][1] == movieID) and x[1][0] > scoreThreshold and x[1][1] > coOccurenceThreshold) # filtering out movie pairs are  that less than 2 i.e. occuring once.
        filteredResults.take(2) # taking 2 elements  from the filered results
        results = filteredResults.map(lambda x: (x[1], x[0])).sortByKey(ascending = False).take(10) # sorting in descending order depending on the similarity scores  and reulting 10 outputs.
        
        print("Top 10 similar movies for " + nameDict[movieID]) # printing the top 10 similar movies with the defined pairs.
        for result in results: # iterating the similar pair of a movie and generates the result
            (sim, pair) = result # gets the similar movie pair from the defined reult
            similarMovieID = pair[0] # assigning the first element of the pair representing the movie id
            if (similarMovieID == movieID): 
                similarMovieID = pair[1] 
            print(nameDict[similarMovieID] + "\tscore: " + str(sim[0]) + "\tstrength: " + str(sim[1]))# it filters the results to only include movies that are similar to a given movie, and prints the top 10 similar movies along with their similarity scores and strengths. The movie ID, score threshold, and co-occurrence threshold are all hard-coded.


Loading movie names...


---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-674749135845352> in <cell line: 1>()
     17         movieID =1
     18         filteredResults = moviePairSimilarities.filter(lambda x: (x[0][0] == movieID or x[0][1] == movieID) and x[1][0] > scoreThreshold and x[1][1] > coOccurenceThreshold)
---> 19         filteredResults.take(2)
     20         results = filteredResults.map(lambda x: (x[1], x[0])).sortByKey(ascending = False).take(10)
     21 

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/rdd.py in take(self, num)
   1923 
   19